<a href="https://colab.research.google.com/github/SeidaAhmed/An-application-of-Machine-Learning-to-COVID-19-data/blob/main/table_template_for_guidance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

IMPORTS


In [ ]:
from bs4 import BeautifulSoup
try:
    import urllib.request as urllib3
    #requests.packages.urllib3.contrib.pyopenssl.extract_from_urllib3()
except ImportError:
    import urllib3
import zlib
import pandas as pd
import numpy as np
import re
import time

Helper functions

In [ ]:
def extract_soup(url):

    hdr = {'User-Agent': 'UbineerCorp info@ubineer.com','Accept-Encoding': 'gzip, deflate','host': 'www.sec.gov'}
    req= urllib3.Request(url,headers= hdr)
    resp = urllib3.urlopen(req, timeout=10)
    decompressed_data=zlib.decompress(resp.read(), 16+zlib.MAX_WBITS)
    soup = BeautifulSoup(
        decompressed_data,
        features="html.parser",
        from_encoding=resp.info().get_param('charset')
        )
    resp.close()

    return(soup)
def openBS(html_file):
    with open(html_file, encoding = "ISO-8859-1") as fp:
        soup = BeautifulSoup(fp,'html.parser')
    return soup

def find_between(s, first, last):
    try:
        start = s.index(first) + len(first)
        end = s.index(last, start)
        return s[start:end]
    except ValueError:
        return ""


def capture_html(ex991,settings):
    open = settings["open"]
    close = settings["close"]

    bs4_8K=extract_soup(ex991)
    test= str(bs4_8K)

    result = find_between(test,open, close)
    html_start = result.find("<")
    html_end = result.rfind(">")
    html = result[html_start:html_end+1]
    return html


def between_section_extract(fullSoup):
    bs4Lst=[]
    allDiv= fullSoup.find_all()

    # print("Here")
    #print(startSoup)
    for ele in range(len(allDiv)):


        if str(allDiv[ele])[:6] == '<table':
            bs4Lst.append(allDiv[ele])

    # master_ele= bs4Lst[0]
    # for item in bs4Lst[1:]:
    #     master_ele.extend(item)
    #print(master_ele)
    return(bs4Lst)





    guidance_bs4=result

    return guidance_bs4

def convert_table(soup):

    df=pd.read_html(str(soup))
    return df

def clean_columns(df):
    min_count =  1
    df = df.dropna( axis=1, thresh=min_count)
    mod_df = df.dropna( axis=0, thresh=min_count)
    return mod_df

def remove_duplicate_columns(df):

    df = df.T.drop_duplicates().T

    return df

def remove_odd_columns(table,num,timelineRows):
    if table.shape[1] >2:
        for row in range(table.shape[0]):
            if table.iat[row,0] == table.iat[row,1] and row not in timelineRows[num]:
                head = table.columns[1]
                print(head)
                table = table.drop(columns = head)
                return table
    return table


def open_close(ex991,settings):
    open = settings["sections"]["guidance"]["params"]["open"]
    close = settings["sections"]["guidance"]["params"]["close"]
    bs4_8K=extract_soup(ex991)
    test= str(bs4_8K)

    result = find_between(test,open, close)
    html_start = result.find("<")
    html_end = result.rfind(">")
    bs4_8K = BeautifulSoup(result,"html.parser")
    section = between_section_extract(bs4_8K)
    return section
def open_table_index(ex991,settings):
    soup = extract_soup(ex991)
    section = soup.find_all('table')[settings["sections"]["guidance"]["params"]["table_index"]]
    return section


Settings functions

In [ ]:
def general_clean(table,num,timelineRows):
    clean_table = clean_columns(table)

    clean_table = remove_duplicate_columns(clean_table)

    clean_table = remove_odd_columns(clean_table,num,timelineRows)
    clean_table.reset_index(drop = True, inplace = True)
    for j in clean_table.index.values:             #remove rows where all values are the same
        row = list(clean_table.loc[j])
        if len(set(row)) == 1 and (j not in timelineRows[num]):
            clean_table = clean_table.drop(index = j)

    clean_table = clean_columns(clean_table)
    for row in clean_table.index.values:
        if pd.isna(clean_table.at[row,0]) and row not in timelineRows[num]:
            clean_table = clean_table.drop(index = row)
    clean_table.columns = range(clean_table.columns.size)
    return clean_table


def keepdupcol_clean(table,num,timelineRows):
    clean_table = clean_columns(table)

    # clean_table = remove_duplicate_columns(clean_table)

    clean_table = remove_odd_columns(clean_table,num,timelineRows)
    clean_table.reset_index(drop = True, inplace = True)

    for j in clean_table.index.values:             #remove rows where all values are the same
        row = list(clean_table.loc[j])
        if len(set(row)) == 1 and (j not in timelineRows[num]):
            clean_table = clean_table.drop(index = j)

    clean_table = clean_columns(clean_table)
    for row in clean_table.index.values:
        if pd.isna(clean_table.at[row,0]) and row not in timelineRows[num]:
            clean_table = clean_table.drop(index = row)
    clean_table.columns = range(clean_table.columns.size)

    return clean_table

def combine_vertical(table,num,timelineRows):
    for row in table.index.values:

        if row not in timelineRows[num]:
            good_values = ''
            row_values = table.loc[row,1:]
            for elem in row_values:
                if not pd.isna(elem):
                    good_values += ' ' + str(elem)
            good_values = good_values.replace('( ','(').replace(' )',')')
            table.at[row,1] = good_values
    table = table.iloc[:,[0,1]]
    return table

def clean_values_vertical(table,num,timelineRows):
    values = table.iloc[:,1]
    for row,value in enumerate(values):
            if isinstance(value,str):
                for i in range(len(value)):
                    if value[i-1].isdigit() and value[i].isalpha():
                        value = value[:i] + " " + value[i:]
                if row not in timelineRows[num]:
                    arr = value.split()
                    vals = []
                    for word in arr:
                        if word.replace(".","").replace(",","").replace("$","").replace('(','').replace(')','').replace('%','').isdigit():
                            vals.append(word)
                    if len(vals) > 2:
                        good_value = value
                        for val in vals[2:]:
                            good_value = good_value.replace(val,'')
                        table.iat[row,1] = good_value


    return table

def format_timeline_vertical(table,num,timelineRows):
    for row in timelineRows[num]:
            if pd.isna(table.at[row,1]):

                table.at[row,1] = table.at[row,0]
            elif pd.isna(table.at[row,0]):
                table.at[row,0] = table.at[row,1]
            elif table.at[row,0] != table.at[row,1]:
                table.at[row,0] = table.at[row,1]
    return table

def make_vertical(table,num,timelineRows):

    table = table.drop(table.columns[2:],axis = 1)
    return table

def split_hyphenated_values(table,num,timelineRows):
    values = table.iloc[:,1:]
    for row in range(len(table.index)):
        if row not in timelineRows[num]:
            for col in range(len(values.columns)):
                value = str(values.iat[row,col]).replace('-',' ')
                table.iat[row,col+1] = value
    return table

def drop_prior(table,num,timelineRows):
  table.columns = range(table.columns.size)
  headers = table.iloc[0].values.tolist()
  # print(headers)
  for index, name in enumerate(headers):
    if type(name) == str and "Prior" in name:
      del table[index]
  table.columns = range(table.columns.size)
  return table

def drop_previous(table,num,timelineRows):
  table.columns = range(table.columns.size)
  headers = table.iloc[0].values.tolist()
  # print(headers)
  for index, name in enumerate(headers):
    if type(name) == str and "Previous" in name:
      del table[index]
  table.columns = range(table.columns.size)
  return table

def drop_previous_rowidx_1(table,num,timelineRows):
  table.columns = range(table.columns.size)
  headers = table.iloc[1].values.tolist()
  # print(headers)
  for index, name in enumerate(headers):
    if type(name) == str and "previous" in name.lower():
      del table[index]
  table.columns = range(table.columns.size)
  return table

def drop_nan_rowidx_1(table,num,timelineRows):
  table.columns = range(table.columns.size)
  headers = table.iloc[1].values.tolist()
  # print(headers)
  for index, name in enumerate(headers):
    if type(name) != str and index != 0:
      del table[index]
  table.columns = range(table.columns.size)
  return table

def drop_original(table,num,timelineRows):
  table.columns = range(table.columns.size)
  headers = table.iloc[0].values.tolist()
  # print(headers)
  for index, name in enumerate(headers):
    if type(name) == str and "Original" in name:
      del table[index]
  table.columns = range(table.columns.size)
  return table

def drop_actual(table,num,timelineRows):
  table.columns = range(table.columns.size)
  headers = table.iloc[0].values.tolist()
  # print(headers)
  for index, name in enumerate(headers):
    if type(name) == str and "Actual" in name:
      del table[index]
  table.columns = range(table.columns.size)
  return table

def drop_unaudited(table,num,timelineRows):
  table.columns = range(table.columns.size)
  headers = table.iloc[0].values.tolist()
  # print(headers)
  for index, name in enumerate(headers):
    if type(name) == str and "unaudited" in name:
      del table[index]
  table.columns = range(table.columns.size)
  return table

def drop_issued(table,num,timelineRows):
  table.columns = range(table.columns.size)
  headers = table.iloc[0].values.tolist()
  # print(headers)
  for index, name in enumerate(headers):
    if type(name) == str and "Issued" in name:
      del table[index]
  table.columns = range(table.columns.size)
  return table

def drop_nan(table,num,timelineRows):
  table.columns = range(table.columns.size)
  headers = table.iloc[0].values.tolist()
  # print(headers)
  for index, name in enumerate(headers):
    if type(name) != str and index != 0:
      del table[index]
  table.columns = range(table.columns.size)
  return table

def drop_last(table,num,timelineRows):
  table.columns = range(table.columns.size)
  del table[table.columns.size - 1]
  table.columns = range(table.columns.size)
  return table

def print_for_debugging(table,num,timelineRows):
  print(table)
  return table

def merge_columns(table,num,timelineRows): #Needs unique row names
  table.columns = range(table.columns.size)
  num_rows = table.shape[0]
  ranges = [list(range(1, 8)), list(range(8, 15))] #first column needs to contain the dates
  toDF = {}
  toDF["Timeline"] = []
  for i in range(1, num_rows):
    toDF[table.iloc[i, 0]] = []
  for j in ranges:
    toDF["Timeline"].append(table.iloc[0, j[0]])
    for k in range(1, num_rows):
      curr_key = table.iloc[k, 0]
      curr_val = ""
      for l in j:
        if str(table.iloc[k, l]) != "nan":
          curr_val += str(table.iloc[k, l])
          curr_val += " "
      toDF[curr_key].append(curr_val)
  newDF = pd.DataFrame.from_dict(toDF, orient='index').reset_index()
  return newDF

def merge_columns_rpd(table,num,timelineRows):
  table.columns = range(table.columns.size)
  num_rows = table.shape[0]
  ranges = [list(range(1, 4)), list(range(4, 7))]
  toDF = {}
  toDF["Timeline"] = []
  for i in range(1, num_rows):
    toDF[table.iloc[i, 0]] = []
  for j in ranges:
    toDF["Timeline"].append(table.iloc[0, j[0]])
    for k in range(1, num_rows):
      curr_key = table.iloc[k, 0]
      curr_val = ""
      for l in j:
        if str(table.iloc[k, l]) != "nan":
          curr_val += str(table.iloc[k, l])
          curr_val += " "
      toDF[curr_key].append(curr_val)
  newDF = pd.DataFrame.from_dict(toDF, orient='index').reset_index()
  return newDF

def rename_rows_rpd(table, num, timelineRows):
  table.iloc[2, 0] = "Annualized Recurring Revenue YOY Growth"
  table.iloc[4, 0] = "Revenue YOY Growth"
  return table

def merge_columns_rpd_2(table,num,timelineRows):
  table.columns = range(table.columns.size)
  num_rows = table.shape[0]
  ranges = [list(range(1, 4)), list(range(4, 7))]
  toDF = {}
  toDF["Timeline"] = []
  for i in range(1, num_rows):
    toDF[table.iloc[i, 0]] = []
  for j in ranges:
    toDF["Timeline"].append(table.iloc[0, j[0]])
    for k in range(1, num_rows):
      curr_key = table.iloc[k, 0]
      curr_val = ""
      for l in j:
        if str(table.iloc[k, l]) != "nan":
          if (curr_val.strip() != str(table.iloc[k, l]).strip()):
            curr_val += str(table.iloc[k, l])
            curr_val += " "
      toDF[curr_key].append(curr_val)
  newDF = pd.DataFrame.from_dict(toDF, orient='index').reset_index()
  return newDF

def merge_columns_rpd_3(table,num,timelineRows):
  table.columns = range(table.columns.size)
  num_rows = table.shape[0]
  ranges = [list(range(1, 7)), list(range(7, 14))]
  toDF = {}
  toDF["Timeline"] = []
  for i in range(1, num_rows):
    toDF[table.iloc[i, 0]] = []
  for j in ranges:
    toDF["Timeline"].append(table.iloc[0, j[0]])
    for k in range(1, num_rows):
      curr_key = table.iloc[k, 0]
      curr_val = ""
      for l in j:
        if str(table.iloc[k, l]) != "nan":
          if (str(table.iloc[k, l - 1]).strip() != str(table.iloc[k, l]).strip()):
            curr_val += str(table.iloc[k, l])
            curr_val += " "
      toDF[curr_key].append(curr_val)
  newDF = pd.DataFrame.from_dict(toDF, orient='index').reset_index()
  return newDF

def drop_row(table, num, timelineRows):
  table = table.drop(1)
  return table

def drop_row_1(table, num, timelineRows):
  if str(table.iloc[1, 0]) == "nan":
    table = table.drop(1)
  return table

def drop_reported_rows(table,num,timelineRows):
  table.index = range(table.index.size)
  labels = table.iloc[:,0].values.tolist()
  for index, name in enumerate(labels):
    if type(name) == str and "reported" in name.lower():
      table.drop(index,axis=0,inplace=True)
  return table

def drop_currency_rows(table,num,timelineRows):
  table.index = range(table.index.size)
  labels = table.iloc[:,0].values.tolist()
  for index, name in enumerate(labels):
    if type(name) == str and "currency" in name.lower():
      table.drop(index,axis=0,inplace=True)
  return table

def split_gaap_nongaap(table,num,timelineRows):
  columns = list(table.iloc[0,:])
  for index,name in enumerate(columns):
    if index==0:
      continue
    if "non-gaap" in name.lower():
      nongaap_column = index
    elif "gaap" in name.lower():
      gaap_column = index
  num_rows = len(table.index)
  new_table = pd.DataFrame([[table.iloc[0,0], table.iloc[0,0]]])
  for i in range(1,num_rows):
    temp_table_gaap = pd.DataFrame([["GAAP " + table.iloc[i,0]  ,table.iloc[i,gaap_column]]])
    temp_table_nongaap = pd.DataFrame([["Non-GAAP " + table.iloc[i,0],table.iloc[i,nongaap_column]]])
    new_table = new_table.append(temp_table_gaap)
    new_table = new_table.append(temp_table_nongaap)
  return new_table

def get_table_conversion_from_cell(tables, settings,coordinates):
  new_tables = tables[:]
  new_table = general_clean(new_tables[0],0,settings["sections"]["guidance"]["format"]["params"]["timelineRows"])


  unit_conv = {"billion": 1000000000, "b": 1000000000, "million": 1000000, "millions": 1000000, "m": 1000000,
             "thousand": 1000, "%": 0.01, "percentage": 0.01, "percent": 0.01, "": 1, "basis points": 0.0001, "bps": 0.0001}
  first_cell = new_table.iloc[coordinates[0]][coordinates[1]]
  if isinstance(first_cell,str):
    first_cell = first_cell.split()
    for word in first_cell:
            if word.replace("(", "").replace(")", "").replace(",","").lower() in unit_conv and word != "":
                settings["sections"]["guidance"]["format"]["params"]["default_unit_conv"] = word.replace("(", "").replace(")", "").replace(",","").lower()

  pass

def addFullYearFiscal(table,num,timelineRows):
  temp = pd.DataFrame([["Full Year Fiscal"]*len(table.axes[1])])
  new_table = pd.concat([temp,table], axis=0)
  return new_table

def addCurrentQuarter(table,num,timelineRows):
  temp = pd.DataFrame([["Current Quarter Guidance"]*len(table.axes[1])])
  new_table = pd.concat([temp,table], axis=0)
  return new_table

def group_columns(table, num, timelineRows): #MADE FOR CRM
  clean_table = table.iloc[:,[0]]
  headers = list(table.iloc[0,:])
  pointer = 0
  groups = [[]]
  for index, name in enumerate(headers):
    if index == 0:
      continue
    if len(groups[pointer])==0:
      groups[pointer].append(index)
      continue
    if name == headers[index-1] or str(name) == "nan":
      groups[pointer].append(index)
    else:
      groups.append([index])
      pointer += 1
  for group in groups:
      temp_table = table.iloc[:,group]
      for row in range(len(temp_table.index.values)):
            if row != 0:
                good_values = ''
                row_values = temp_table.iloc[row,:]
                if len(set(row_values)) == 1:
                  good_values = list(row_values)[0]
                else:
                  for elem in row_values:
                      if not pd.isna(elem):
                          good_values += ' ' + str(elem)
                  good_values = good_values.replace('( ','(').replace(' )',')')
                temp_table.iat[row,0] = good_values
      temp_table = temp_table.iloc[:,[0]]
      lst = [clean_table,temp_table]
      clean_table = pd.concat(lst,axis=1)
  return clean_table


DATAFRAME TO DICT

In [ ]:
def dataframe_to_dict(df):
    if df.empty:
      return
    fst_col=list(df. iloc[:, 0])
    myDict = {}
    fst_row=list(df.iloc[0])
    mylist=[]
    # If tables are joint (ie. col 2 and col 3 actually represent the lower and
    # upper of a variable instead of 2 different reports)
    if fst_row[-1] == fst_row[-2]:  #change to orientation == vertical
        for i in range(len(fst_col)):
          row = list(df.iloc[i])
          # print("row")
          # print(row)
          #if last 2 elems of row are equal and starts with a letter, then it is
          # the start of new "period", with label row[-1]
          if row[-1] == row[-2] and row[-1][0].isalpha(): # header row
            if myDict:
              #reset dictionary
              dictionary_copy = myDict.copy()
              mylist.append(dictionary_copy)
              dict.clear(myDict)
            myDict["Timeline"] = row[-1].replace(':','')
          else:
            if len(row) >= 3 and isinstance(row[2],str): # if there is a upper bound
              myDict[row[0]] = str(row[1]) + " - " + str(row[2])
            else: # one value
              myDict[row[0]] = row[1]

        dictionary_copy = myDict.copy()
        mylist.append(dictionary_copy)
    else:   #change to orientation == horizontal
    # Otherwise, table is not joint
      for i in range(1,len(fst_row)):
          dict.clear(myDict)

          myDict["Timeline"] = fst_row[i].replace('Guidance','')
          for k in range(1,len(df. iloc[:, 0])):
              data_row=list(df.iloc[k])
              if str(data_row[i]) != 'nan' and str(data_row[i]) != 'Guidance' :
                  myDict[fst_col[k]] = str(data_row[i])


          dictionary_copy = myDict.copy()
          mylist.append(dictionary_copy)
    return mylist

Universal capture guidance function

In [ ]:
def parseGuidance_table(ex991,settings):
    functions = {"table_open_close":open_close,"from_index":open_table_index,"combine_vertical":combine_vertical,
                "general_clean":general_clean, "combine_vertical":combine_vertical,"clean_values_vertical":clean_values_vertical, "make_vertical":make_vertical,
                "format_timeline_vertical":format_timeline_vertical,"split_hyphenated_values":split_hyphenated_values, "drop_prior": drop_prior, "drop_previous": drop_previous, "drop_original": drop_original, "print" : print_for_debugging, "drop_nan" : drop_nan, "drop_actual" : drop_actual, "drop_unaudited" : drop_unaudited, "drop_issued" : drop_issued, "merge_columns" : merge_columns, "keepdupcol" : keepdupcol_clean, "drop_last" : drop_last, "merge_rpd" : merge_columns_rpd, "rename_rows_rpd" : rename_rows_rpd, "merge_rpd_2" : merge_columns_rpd_2, 'drop_row_1' : drop_row, "merge_rpd_3" : merge_columns_rpd_3, "drop_previous_rowidx_1" : drop_previous_rowidx_1, "drop_nan_rowidx_1" : drop_nan_rowidx_1, "drop_row_1" : drop_row_1,
                 "get_table_conversion_from_cell":get_table_conversion_from_cell, "addCurrentQuarter":addCurrentQuarter,"split_gaap_nongaap":split_gaap_nongaap, "group_columns":group_columns
                }

    capturefunction = functions[settings["sections"]["guidance"]["selection_method"]]
    try:
        section = capturefunction(ex991,settings)
    except Exception as e:
        print(f"Unable to capture guidance: {ex991}")
        print(e)
        return None
    tables = convert_table(section)
    get_default_conv_function = settings["sections"]["guidance"]["format"]["params"].get("unit_conv_function", None)
    if get_default_conv_function:
        functions[list(get_default_conv_function.keys())[0]](tables, settings,list(get_default_conv_function.values())[0])
    select_index = settings["sections"]["guidance"]["params"].get("select_index",list(range(len(tables))))
    timelineRows = settings["sections"]["guidance"]["format"]["params"]["timelineRows"]
    count = -1
    for table_num, table in enumerate(tables):
      if table_num in select_index:
        count += 1
        for function in settings["sections"]["guidance"]["format"]["functions"]:
            print(function)
            function = functions[function]
            table = function(table,count,timelineRows)
            display(table)
            tables[table_num] = table
    data = []
    for table_num, table in enumerate(tables):
      if table_num in select_index:
        # print(table)
        data += dataframe_to_dict(table)
    return data

SETTINGS

In [ ]:
settings = {
    "sections": {
        "guidance": {
            "selection_method": "table_open_close", #other method is "from_index"
            "params": {
                "open": "CUSTOMER COUNT METRICS",
                "close": "(a) Direct Sales Customers are customers that were sold through our direct sales force and channel partners.",
                #"select_index":[0],
               "table_index": slice(1) #if we want to capture the 2 first tables using the index
            },
            "format": {
                "format_type": "table", #always table for table companies
                "functions": [


                    "general_clean"

                ],
                "params": {
                    "timelineRows": [
                        [
                            0
                        ],

                    ],

                    "default_unit_conv":"billions",
                }
            }
        }
    }
}
eight_k =[

]

# eight_k=['https://www.sec.gov/Archives/edgar/data/707549/000070754920000167/lrcxexhibit991q12021.htm']

eight_k=['https://www.sec.gov/Archives/edgar/data/1441816/000144181625000046/mdb-13125xex991xrelease.htm']

In [ ]:
unit_conv = unit_conv={"billion":1000000000, "billions":1000000000, "million":1000000,"millions":1000000,"thousand":1000,"%":0.01,"percentage":0.01,"percent":0.01,"":1,"basis-points":0.0001,"bps":0.0001,"Million":1000000,"Billion":1000000000}
years = []
for i in range(2000,2100):
  years.append(i)
def unitConversion(insert_lst):
  value = None
  insert_lst[0] = insert_lst[0].replace('flat','0%')
  unit_conv={"b":1000000000,"m":1000000,"billion":1000000000, "billions":1000000000, "million":1000000,"millions":1000000,"thousand":1000,"%":0.01,"percentage":0.01,"percent":0.01,"":1,"basis-points":0.0001,"bps":0.0001,"Million":1000000,"Billion":1000000000}
  if len(insert_lst) != 2:

    for unit in list(unit_conv.keys()):
      if unit == '':
        pass
      elif unit in insert_lst[0]:
        insert_lst = [str(insert_lst[0]).replace(unit,''), unit]



  insert_lst[-1] =insert_lst[-1].lower()


  insert_lst[0] = str(insert_lst[0]).replace(',','').replace('$','').replace('(','-').replace(')','').replace('%','').strip()
  #Andi adds brackets


  insert_lst[0] = float(insert_lst[0])

  for ele in unit_conv.keys():
    #print(ele)
    if ele == insert_lst[-1]:
      value = insert_lst[0] * unit_conv[ele]
      #attempt to fix floating point error
      value = round(value,4)
    else:
      pass
  if value == None:
    print("Value is None: ", insert_lst)
  return value
def process(raw_info_dict, default_conv): #If you ever find an error with this function let @Maxime know ASAP
    temp_dict = {}

    if len(raw_info_dict)>1:
        temp_dict = {}
        temp_dict["Timeline"] = raw_info_dict["Timeline"]

        for key, value in raw_info_dict.items():
          if not isinstance(key,str):
            continue
          if not isinstance(value,str):
            print(f"Value for {key} is nan")
            continue
            # Edge case where number and conversion is in the same word (i.e. "350M")
          for i in range(1, len(value)):
              if value[i-1].isdigit() and value[i].isalpha():
                  value = value[:i] + " " + value[i:]

          value = re.sub(r'(\d+)(\%)',r'\1 \2', value )
          value = value.replace("basis points", "basis-points")
          value = value.replace("low-teens", "11 % 14 %")
          arr = value.split()
          prefix = 1
          vals = []
          if (len(arr) == 0 or key=="Timeline"):
            continue
          conversions = [] #case where there are multiple conversions in the same captured data
          conversion = ''
          #Find unit conversion
          for i, elem in enumerate(arr):
            if elem == 'breakeven' or elem == 'break-even' or elem == 'zero': #replace words meaning 0
              arr[i] = '0'
            if elem in unit_conv.keys():
              conversion = elem
              conversions.append(conversion)
          # Check if unit conversion is in key
          if conversion == '':
            for word in key.split():
              if word.replace("(","").replace(")","") in unit_conv.keys(): #Replace unit conversion in key, ie. if key is "Revenue (in millions)", it should be "Revenue"
                conversion = word.replace("(","").replace(")","")
                conversions.append(conversion)
          if conversion == '' and default_conv and ("per share" not in key.lower() and "eps" not in key.lower() and "per diluted share" not in key.lower()):
              conversions.append(default_conv)

          if len(conversions) == 0:
            conversions.append('')

          plus_minus_flag = 'plus minus' in value or '+/-' in value
          if plus_minus_flag and (conversions == ['%'] or conversions == ['percent']):
            if 'flat' not in value:
              conversions.insert(0,'')
            else:
              conversions.append(conversions[0])
          if len(set(conversions)) > 1:
            multi = True
          else:
            multi = False


          # regex finds () and [] brackets
          key = re.sub("\(.*?\)|\[.*?\]","", key)
          #Check if numbers should be negative
          if ("loss" in arr) or ("negative" in arr) or ("decrease" in arr):
            prefix = -1
          conv_num = 0
          #Find values (should be 1 or 2)


          # IMPROVED HANDLING OF NEGATIVE AND POSITIVE VALS IN SAME CHUNK - by Jack Li
          ignore_set = ['$']
          switch = False
          negative = False
          state = 0
          for elem in arr:
            if "(" in elem and state == 0:
              state = 1
            if (elem.replace(".","").replace(",","").replace("$","").replace('(','').replace(')','').replace('%','').isdigit() and state == 1) or (elem == 'loss' and state == 1):
              state = 2
            if ")" in elem and state == 1:
              state = 0
            if ")" in elem and state == 2:
              switch = True
              break
          pos_search = False

          if switch is False:
            for elem in arr:
              if elem == 'positive':
                pos_search = True
                continue
              if pos_search:
                if (not elem.replace(".","").replace(",","").replace("$","").replace('(','').replace(')','').replace('%','').replace('-','').replace('~','').isdigit()) and (elem not in ignore_set):
                  pos_search = False

              if elem.replace(".","").replace(",","").replace("$","").replace('(','').replace(')','').replace('%','').replace('-','').replace('~','').isdigit():
                if pos_search:
                  vals.append(unitConversion([elem, conversions[conv_num]]))
                  if multi: conv_num += 1
                  pos_search = False
                else:
                  vals.append(unitConversion([elem, conversions[conv_num]])*prefix)
                  if multi: conv_num += 1
          else:
            for elem in arr:
              if "(" in elem:
                negative = True
              if ")" in elem:
                negative = False
              if elem.replace(".","").replace(",","").replace("$","").replace('(','').replace(')','').replace('%','').replace('-','').replace('~','').isdigit():
                if (negative):
                  vals.append(unitConversion([elem, conversions[conv_num]])*-1)
                  if multi: conv_num += 1
                else:
                  vals.append(unitConversion([elem, conversions[conv_num]]))
                  if multi: conv_num += 1
          if len(vals) == 0:
            print(key+ " is empty")
          if max(vals)-min(vals) > 100 * unit_conv[conversion]:
            for val in vals:
              if val/unit_conv[conversions[0]] in years:
                  vals.remove(val)
                  break
          if plus_minus_flag:
            if max(vals)-min(vals) > 100 * unit_conv[conversions[0]]:
              for val in vals:
                if abs(val/unit_conv[conversions[0]]) in years:
                    vals.remove(val)
                    break
          elif max(vals)-min(vals) > 100 * unit_conv[conversion]:
            for val in vals:
              if abs(val/unit_conv[conversion]) in years:
                  vals.remove(val)
                  break
          if plus_minus_flag:
            mean_val = vals[0]
            delta = None
            for val in vals:
              if val !=mean_val:
                delta = val
            if not delta:
              delta = vals[1]
            if (conversions.count('%') + conversions.count('percent')) + conversions.count('basis-points') + conversions.count('bps') == 1:
              vals =  [None,None]
              vals[0] = round(mean_val*(1+delta),4)
              vals[1] = round(mean_val*(1-delta),4)
            else:
              vals = [None,None]
              vals[0] = round(mean_val + delta,4)
              vals[1] = round(mean_val - delta,4)
          temp_dict[key+" Upper"] = max(vals)
          temp_dict[key+" Lower"] = min(vals)

    return temp_dict
def clean(raw_info,settings):
    Outlook = []
    for i in range(len(raw_info)):
        if process(raw_info[i],settings["sections"]["guidance"]["format"]["params"].get("default_unit_conv", None)) != None:
            Outlook.append(process(raw_info[i],settings["sections"]["guidance"]["format"]["params"].get("default_unit_conv", None)))
    return Outlook

import json
for link in eight_k:
  outlook = []
  try:
    print(link)
    data = parseGuidance_table(link,settings)
    data = clean(data,settings)
    for dct in data:
      print(json.dumps(dct,indent=4))
  except Exception as e:
    print(f"not captured {link}")
    print(e)

https://www.sec.gov/Archives/edgar/data/1441816/000144181625000046/mdb-13125xex991xrelease.htm


<ipython-input-2-f6d58fcb6e2b>:71: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df=pd.read_html(str(soup))


general_clean


,0,1,2,3,4,5,6,7,8,9
0,NaN,1/31/2023,4/30/2023,7/31/2023,10/31/2023,1/31/2024,4/30/2024,7/31/2024,10/31/2024,1/31/2025
1,Total Customers (a),"40,800+","43,100+","45,000+","46,400+","47,800+","49,200+","50,700+","52,600+","54,500+"
2,Direct Sales Customers(b),"6,400+","6,700+","6,800+","6,900+","7,000+","7,100+","7,300+","7,400+","7,500+"
3,MongoDB Atlas Customers,"39,300+","41,600+","43,500+","44,900+","46,300+","47,700+","49,200+","51,100+","53,100+"
4,Customers over $100K(c),1651,1761,1855,1972,2052,2137,2189,2314,2396


general_clean
not captured https://www.sec.gov/Archives/edgar/data/1441816/000144181625000046/mdb-13125xex991xrelease.htm
list index out of range


In [ ]:
def process(raw_info_dict, default_conv): #If you ever find an error with this function let @Maxime know ASAP
    temp_dict = {}
    print('Raw Info')
    print(raw_info_dict)
    print('Default Info')
    print(default_conv)
    if len(raw_info_dict)>1:
        temp_dict = {}
        temp_dict["Timeline"] = raw_info_dict["Timeline"]

        for key, value in raw_info_dict.items():
          if not isinstance(key,str):
            continue
          if not isinstance(value,str):
            print(f"Value for {key} is nan")
            continue
            # Edge case where number and conversion is in the same word (i.e. "350M")
          for i in range(1, len(value)):
              if value[i-1].isdigit() and value[i].isalpha():
                  value = value[:i] + " " + value[i:]

          value = re.sub(r'(\d+)(\%)',r'\1 \2', value )
          value = value.replace("basis points", "basis-points")
          value = value.replace("low-teens", "11 % 14 %")
          arr = value.split()
          print(arr)
          prefix = 1
          vals = []
          if (len(arr) == 0 or key=="Timeline"):
            continue
          conversions = [] #case where there are multiple conversions in the same captured data
          conversion = ''
          #Find unit conversion
          for i, elem in enumerate(arr):
            print("i: ", i, "elem: ",elem)
            if elem == 'breakeven' or elem == 'break-even' or elem == 'zero' or elem == "$—" : #replace words meaning 0
              arr[i] = '0'
            if elem in unit_conv.keys():
              conversion = elem
              print("conversion", conversion)
              conversions.append(conversion)
              print("conversions", conversions)
          # Check if unit conversion is in key
          if conversion == '':
            for word in key.split():
              if word.replace("(","").replace(")","") in unit_conv.keys(): #Replace unit conversion in key, ie. if key is "Revenue (in millions)", it should be "Revenue"
                conversion = word.replace("(","").replace(")","")
                conversions.append(conversion)
          if conversion == '' and default_conv and ("per share" not in key.lower() and "eps" not in key.lower() and "per diluted share" not in key.lower()):
              conversions.append(default_conv)

          if len(conversions) == 0:
            conversions.append('')

          plus_minus_flag = 'plus minus' in value or '+/-' in value
          if plus_minus_flag and (conversions == ['%'] or conversions == ['percent']):
            if 'flat' not in value:
              conversions.insert(0,'')
            else:
              conversions.append(conversions[0])
          if len(set(conversions)) > 1:
            multi = True
          else:
            multi = False


          # regex finds () and [] brackets
          key = re.sub("\(.*?\)|\[.*?\]","", key)


          #Check if numbers should be negative
          if ("loss" in arr) or ("negative" in arr) or ("decrease" in arr):
            prefix = -1
          conv_num = 0
                    # Improved percentage handling

          # IMPROVED HANDLING OF NEGATIVE AND POSITIVE VALS IN SAME CHUNK - by Jack Li
          ignore_set = ['$']
          switch = False
          negative = False
          state = 0
          for elem in arr:
            if "(" in elem and state == 0:
              state = 1
              print('state 1 trigger')
            if (elem.replace(".","").replace(",","").replace("$","").replace('(','').replace(')','').replace('%','').isdigit() and state == 1) or (elem == 'loss' and state == 1):
              state = 2
              print('state 2 trigger')
            if ")" in elem and state == 1:
              state = 0
              print('state 0 trigger')
            if ")" in elem and state == 2:
              switch = True
              break
          pos_search = False
          print('Switch:', switch)

          if switch is False:
            for elem in arr:
              if elem == 'positive':
                pos_search = True
                continue
              if pos_search:
                if (not elem.replace(".","").replace(",","").replace("$","").replace('(','').replace(')','').replace('%','').replace('-','').replace('~','').isdigit()) and (elem not in ignore_set):
                  pos_search = False

              if elem.replace(".","").replace(",","").replace("$","").replace('(','').replace(')','').replace('%','').replace('-','').replace('~','').isdigit():
                if pos_search:
                  print('First if statement: unitConversion', unitConversion([elem, conversions[conv_num]]))
                  vals.append(unitConversion([elem, conversions[conv_num]]))
                  if multi: conv_num += 1
                  pos_search = False
                else:
                  print('Second if statement unitConversion:', unitConversion([elem, conversions[conv_num]])*prefix)
                  vals.append(unitConversion([elem, conversions[conv_num]])*prefix)

                  if multi: conv_num += 1
          else:

            for elem in arr:
              if "(" in elem:

                negative = True
              if ")" in elem:
                negative = False

              if elem.replace(".","").replace(",","").replace("$","").replace('(','').replace(')','').replace('%','').replace('-','').replace('~','').isdigit():
                if (negative):

                  vals.append(unitConversion([elem, conversions[conv_num]])*-1)
                  if multi: conv_num += 1
                else:
                  print('************')
                  print('ELEM')
                  print(elem)
                  print('VALS')
                  print(vals)
                  if ('%' in elem) and (')' in elem )and ('(' in elem):
                    print("Test else")
                    print('^^^^^^^^^^^^^^^^^^^^^^^^^^^^')
                    if elem.replace(".","").replace(",","").replace("$","").replace('(','').replace(')','').replace('%','').replace('-','').replace('~','').isdigit():
                      print(elem)
                      vals.append(unitConversion([elem.replace(".","").replace(",","").replace("$","").replace('(','').replace(')','').replace('%','').replace('-','').replace('~',''), '%'])*-1)
                  else:

                    vals.append(unitConversion([elem, conversions[conv_num]]))
                  print("+++++++++++++")
                  print(vals)
                  if multi: conv_num += 1
          if len(vals) == 0:
            print(key+ " is empty")
          if max(vals)-min(vals) > 100 * unit_conv[conversion]:
            for val in vals:
              print(val)
              if val/unit_conv[conversions[0]] in years:
                  vals.remove(val)
                  break
          if plus_minus_flag:
            if max(vals)-min(vals) > 100 * unit_conv[conversions[0]]:
              for val in vals:
                if abs(val/unit_conv[conversions[0]]) in years:
                    vals.remove(val)
                    break
          elif max(vals)-min(vals) > 100 * unit_conv[conversion]:
            for val in vals:
              if abs(val/unit_conv[conversion]) in years:
                  vals.remove(val)
                  break
          if plus_minus_flag:
            mean_val = vals[0]
            delta = None
            for val in vals:
              if val !=mean_val:
                delta = val
            if not delta:
              delta = vals[1]

            if (conversions.count('%') + conversions.count('percent')) + conversions.count('basis-points') + conversions.count('bps') == 1:
              vals =  [None,None]
              vals[0] = round(mean_val*(1+delta),4)
              vals[1] = round(mean_val*(1-delta),4)
            else:
              vals = [None,None]
              vals[0] = round(mean_val + delta,4)
              vals[1] = round(mean_val - delta,4)
          temp_dict[key+" Upper"] = max(vals)
          temp_dict[key+" Lower"] = min(vals)

    return temp_dict

def clean(raw_info,settings):
  Outlook = []
  for i in range(len(raw_info)):
      if process(raw_info[i],settings["sections"]["guidance"]["format"]["params"].get("default_unit_conv", None)) != None:
          Outlook.append(process(raw_info[i],settings["sections"]["guidance"]["format"]["params"].get("default_unit_conv", None)))
  return Outlook

import json
for link in eight_k:
  outlook = []
  try:
    print(link)
    data = parseGuidance_table(link,settings)
    data = clean(data,settings)
    for dct in data:
      print(json.dumps(dct,indent=4))
  except Exception as e:
    print(f"not captured {link}")
    print(e)

https://www.sec.gov/Archives/edgar/data/1441816/000144181625000046/mdb-13125xex991xrelease.htm
general_clean


<ipython-input-2-f6d58fcb6e2b>:71: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df=pd.read_html(str(soup))


,0,1,2,3,4,5,6,7,8,9
0,NaN,1/31/2023,4/30/2023,7/31/2023,10/31/2023,1/31/2024,4/30/2024,7/31/2024,10/31/2024,1/31/2025
1,Total Customers (a),"40,800+","43,100+","45,000+","46,400+","47,800+","49,200+","50,700+","52,600+","54,500+"
2,Direct Sales Customers(b),"6,400+","6,700+","6,800+","6,900+","7,000+","7,100+","7,300+","7,400+","7,500+"
3,MongoDB Atlas Customers,"39,300+","41,600+","43,500+","44,900+","46,300+","47,700+","49,200+","51,100+","53,100+"
4,Customers over $100K(c),1651,1761,1855,1972,2052,2137,2189,2314,2396


general_clean
not captured https://www.sec.gov/Archives/edgar/data/1441816/000144181625000046/mdb-13125xex991xrelease.htm
list index out of range
